In [6]:
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.tools.visualization import dag_drawer
from cutting_help_fun import *
import pickle
import networkx as nx

circ = pickle.load(open('results/supremacy_circuit_4_8.dump', 'rb' ))
original_dag = circuit_to_dag(circ)
q = circ.qregs[0]
# positions = [(q[2], 1), (q[7], 1), (q[8],3), (q[9],6), (q[10], 4), (q[10],1), (q[14], 1)]
positions = [(q[2], 1), (q[7], 1), (q[10],1), (q[14], 1)]
wires_being_cut = [x[0] for x in positions]

cut_dag, path_order_dict = cut_edges(original_dag=original_dag, positions=positions)
dag_drawer(cut_dag, filename='results/cut_dag.pdf')
print('cut_dag has %d connected components' % nx.number_weakly_connected_components(cut_dag._multi_graph))

# print('\npath_order_dict:')
# for key in path_order_dict:
#     print(key, path_order_dict[key])

in_out_arg_dict = contains_wire_nodes(cut_dag)
# print('\nin_out_arg_dict:')
# for key in in_out_arg_dict:
#     print(key, in_out_arg_dict[key])

sub_reg_dicts, input_wires_mapping = sub_circ_reg_counter(cut_dag, in_out_arg_dict)
# print('\nsub_reg_dicts:')
# for reg_dict in sub_reg_dicts:
#     print(reg_dict)

# print('\ninput_wires_mapping:')
# [print(x, input_wires_mapping[x]) for x in input_wires_mapping]

components = list(nx.weakly_connected_components(cut_dag._multi_graph))
translation_dict = translation_dict_calc(input_wires_mapping, components, in_out_arg_dict, sub_reg_dicts)
# print('\ntranslation_dict:')
# [print(x, translation_dict[x]) for x in translation_dict]

cut_dag has 2 connected components


In [7]:
# complete_path_map = complete_path_calc(path_order_dict, input_wires_mapping, translation_dict, sub_reg_dicts)
# print('\ncomplete_path_map:')
# [print(x, complete_path_map[x]) for x in complete_path_map]

sub_circs = generate_sub_circs(cut_dag, positions)
for idx, sub_circ in enumerate(sub_circs):
    sub_circ.draw(output='text',line_length = 400, filename='results/sub_circ_%d.txt' % idx)
    dag_drawer(circuit_to_dag(sub_circ), filename='results/sub_dag_%d.pdf' % idx)

Begin component  0
reg_dict:  {'q': QuantumRegister(10, 'q'), 'ancilla_q': QuantumRegister(3, 'ancilla_q'), 'measure_q': ClassicalRegister(1, 'measure_q')}
added registers in the sub circuit: [QuantumRegister(10, 'q'), QuantumRegister(3, 'ancilla_q')] [ClassicalRegister(1, 'measure_q')]
finished component 0

Begin component  1
reg_dict:  {'measure_q': ClassicalRegister(3, 'measure_q'), 'q': QuantumRegister(6, 'q'), 'ancilla_q': QuantumRegister(1, 'ancilla_q')}
added registers in the sub circuit: [QuantumRegister(6, 'q'), QuantumRegister(1, 'ancilla_q')] [ClassicalRegister(3, 'measure_q')]
finished component 1

